# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [24]:
class Bmatrix:  ##set a super class as abstraction
    def __init__(self, n=None, m=None, values=None): #none allows the class to be initialized in 2 ways
       if values is not None:
            # Matrix with list of lists
            if not all(len(row) == len(values[0]) for row in values):
                raise ValueError("All rows and columns must be the same number")
            self.matrix = values
            self.n = len(values)       #  rows
            self.m = len(values[0])    #  columns
       elif n is not None and m is not None:
            # Matrix will fill with zeros
            self.n = n
            self.m = m
            self.matrix = [[0 for _ in range(m)] for _ in range(n)]
       else:
            raise ValueError("Must provide either dimensions or a list of lists")
           
    def __repr__(self):
        return '\n'.join(['\t'.join([str(elem) for elem in row]) for row in self.matrix])

    def __getitem__(self, key):
        # Allow M[i,j] access
        if isinstance(key, tuple):
            i, j = key
            return self.matrix[i][j]
        # Allow M[i] access
        return self.matrix[key]

    def __setitem__(self, key, value):
        # Allow M[i,j] assignment
        if isinstance(key, tuple):
            i, j = key
            self.matrix[i][j] = value
        # Allow M[i] assignment
        else:
            self.matrix[key] = value

    def __same_dimension__(self, other):
        # Check if two matrices are equal (same size and same elements)
        if isinstance(other, Bmatrix):
            return self.matrix == other.matrix
        return False

    def __assign__(self, other):
        # Assignment from another matrix or a list of lists
        if isinstance(other, Bmatrix):
            if self.n == other.n and self.m == other.m:
                self.matrix = [row[:] for row in other.matrix]
            else:
                raise ValueError("Matrices must be the same size")
        elif isinstance(other, list):
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.matrix = [row[:] for row in other]
            else:
                raise ValueError("List of lists must match matrix size")
        else:
            raise ValueError("Invalid assignment type") 

In [25]:
M1 = Bmatrix(3, 3)  # matrix initialized by 0s
M2 = Bmatrix(values=[[1, 2, 3], [4, 5, 6], [5, 3, 1]])  # 3x3 matrix from list of lists

# Access matrix elements
print(M1[0, 0])  # which element is at (0,0)?
M1[0, 0] = 4   # Set element at (0,0)
print(M1.matrix)

# Modify assign M2 to M1
M1.__assign__(M2)
print(M1.matrix)

# Assign a list of lists to M1
M1.__assign__([[7, 8, 9], [10, 11, 12], [ 6, 3, 7]])
print(M1.matrix)

0
[[4, 0, 0], [0, 0, 0], [0, 0, 0]]
[[1, 2, 3], [4, 5, 6], [5, 3, 1]]
[[7, 8, 9], [10, 11, 12], [6, 3, 7]]


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [26]:
class matrix(Bmatrix):
    def shape(self):
        # tuple
        return (self.n, self.m)

    def transpose(self):
        # new matrix will be transposed
        transposed = [[self.matrix[j][i] for j in range(self.n)] for i in range(self.m)]
        return matrix(values=transposed)

    def row(self, n):
        # Return the nth row as a new matrix object
        if n >= self.n or n < 0:
            raise IndexError("Row index out of range")
        return matrix(values=[self.matrix[n]])
    def column(self, n):
        if n >= self.m or n < 0:
            raise IndexError("Column index out of range")
        col = [[self.matrix[i][n]] for i in range(self.n)]
        return matrix(values=col)

    def to_list(self):
        # makes matrix into a list of lists
        return [row[:] for row in self.matrix]

    def block(self, n_0, n_1, m_0, m_1):
        # Return a smaller matrix from rows n_0 to n_1 and columns m_0 to m_1
        if not (0 <= n_0 <= n_1 < self.n and 0 <= m_0 <= m_1 < self.m):
            raise IndexError("The indices out of range")
        block_matrix = [row[m_0:m_1+1] for row in self.matrix[n_0:n_1+1]] ##not including index n_1
        return matrix(values=block_matrix)


In [27]:
M1 = matrix(values=[[1, 4, 3], [4, 4, 6], [7, 4, 9]])

print("Shape:", M1.shape())  

# Transpose the matrix
transpose_M1 = M1.transpose()
print("Transpose:", transpose_M1.to_list())  # Output: [[1, 4, 7], [2, 5, 8], [3, 6, 9]]

# Get the first row
print("Row 0:", M1.row(0).to_list())  # Output: [[1, 2, 3]]

# Get the second column
print("Column 1:", M1.column(1).to_list())  # Output: [[2], [5], [8]]

# Get a block of the matrix
block_M1 = M1.block(1, 2, 1, 2)
print("Block from (1,2) to (1,2):", block_M1.to_list())  # Output: [[5, 6], [8, 9]]


Shape: (3, 3)
Transpose: [[1, 4, 7], [4, 4, 4], [3, 6, 9]]
Row 0: [[1, 4, 3]]
Column 1: [[4], [4], [4]]
Block from (1,2) to (1,2): [[4, 6], [4, 9]]


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [28]:
#1
def constant(n, m, c):
    return [[float(c) for _ in range(m)] for _ in range(n)]

In [29]:
print(constant(4, 3, 5.5))

[[5.5, 5.5, 5.5], [5.5, 5.5, 5.5], [5.5, 5.5, 5.5], [5.5, 5.5, 5.5]]


In [30]:
#2
def zeros(n, m):
    return constant(n, m, 0.0)

In [31]:
print(zeros(2,3))

[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]


In [32]:
#3
def ones(n, m):
    return constant(n, m, 1.0)

In [33]:
print(ones(3,3))

[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]


In [34]:
#4
def eye(n):
    return [[float(1) if i == j else float(0) for j in range(n)] for i in range(n)]

In [35]:
print(eye(3))

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [40]:
class Bmatrix(matrix):
    def shape(self):
        return (self.n, self.m)
    def transpose(self):
        return Bmatrix(values=[[self.matrix[j][i] for j in range(self.n)] for i in range(self.m)])
    def row(self, n):
        if n < 0 or n >= self.n:
            raise IndexError("Row index out of range")
        return Bmatrix(values=[self.matrix[n]])
    def column(self, n):
        if n < 0 or n >= self.m:
            raise IndexError("Column index out of range")
        return Bmatrix(values=[[self.matrix[i][n] for i in range(self.n)]])
    def to_list(self):
        return self.matrix
    def block(self, n_0, n_1, m_0, m_1):
        if n_0 < 0 or n_1 > self.n or m_0 < 0 or m_1 > self.m:
            raise IndexError("Block indices out of range")
        return Bmatrix(values=[row[m_0:m_1] for row in self.matrix[n_0:n_1]])

    def scalarmul(self, c):
        # multiply element by c
        return Bmatrix(values=[[c * element for element in row] for row in self.matrix])

    def add(self, other):
        if not isinstance(other, Bmatrix):
            raise ValueError("Can only add another Bmatrix")
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same dimensions to add") ##check
        return Bmatrix(values=[[self.matrix[i][j] + other.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, other):
        if not isinstance(other, Bmatrix):
            raise ValueError("Can only subtract another Bmatrix")
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same dimensions to subtract")
        return Bmatrix(values=[[self.matrix[i][j] - other.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, other):
        #product of two matrices
        if not isinstance(other, Bmatrix):
             raise ValueError("Can only multiply by another Bmatrix")
        if self.m != other.n:
            raise ValueError("Number of columns in the first matrix must be equal to the number of rows in the second matrix")
        
        result_matrix = [[sum(self.matrix[i][k] * other.matrix[k][j] for k in range(self.m)) for j in range(other.m)] for i in range(self.n)]
        
        return Bmatrix(values=result_matrix)
        
    def element_mult(self, other):
        #multiply corresponding elements of two matrices
        if not isinstance(other, Bmatrix):
            raise ValueError("Can only multiply by another Bmatrix")
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same dimensions!")
        return Bmatrix(values=[[self.matrix[i][j] * other.matrix[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, other):#returns true is both matrices are the same shape 
        if not isinstance(other, Bmatrix):
            return False
        return self.shape() == other.shape() and all(self.matrix[i] == other.matrix[i] for i in range(self.n))



In [41]:
first = Bmatrix(values=[[2, 2, 3], [4, 9, 6], [7, 8, 10]])
second = Bmatrix(values=[[10, 8, 7], [6, 4, 4], [3, 3, 3]])
###create more matrcies using thexe functions:
a = first.scalarmul(2)
print("Scalar Multiplication:\n", a.matrix)
b = first.add(second)
print("Matrix Addition:\n", b.matrix)
c = first.sub(second)
print("Matrix Subtraction:\n", c.matrix)
d = first.element_mult(second)
print("Element-wise Multiplication:\n", e.matrix)
is_equal = first.equals(second)
print("Matrices A and B are equal:", is_equal)


Scalar Multiplication:
 [[4, 4, 6], [8, 18, 12], [14, 16, 20]]
Matrix Addition:
 [[12, 10, 10], [10, 13, 10], [10, 11, 13]]
Matrix Subtraction:
 [[-8, -6, -4], [-2, 5, 2], [4, 5, 7]]
Element-wise Multiplication:
 [[20, 16, 21], [24, 36, 24], [21, 24, 30]]
Matrices A and B are equal: False


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [42]:
first = Bmatrix(values=[[2, 2, 3], [4, 9, 6], [7, 8, 10]])
second = Bmatrix(values=[[10, 8, 7], [6, 4, 4], [3, 3, 3]])
# Test operation
result1 = first.mat_mult(second) #M*N
print("M * N M1:\n", result1)

M * N M1:
 41	33	31
112	86	82
148	118	111


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [43]:
### these Properties are: Associative, Distributive, Not commutative multiplication and 
# multiplying by the identity matrix.

In [44]:
A = Bmatrix(values=[[2, 2], [3, 3]])
B = Bmatrix(values=[[2, 6], [4, 8]])
C = Bmatrix(values=[[4, 5], [3, 12]])
I = Bmatrix(values=[[1, 9], [0, 8]])

In [45]:
# Associative:
left_assoc = A.mat_mult(B).mat_mult(C)  # (AB)C
right_assoc = A.mat_mult(B.mat_mult(C)) # A(BC)
print("(AB)C = A(BC):\n", left_assoc, "\n", right_assoc, "\n")


(AB)C = A(BC):
 132	396
198	594 
 132	396
198	594 



In [46]:
# Matrix multiplication is not communitative
AB = A.mat_mult(B)
BA = B.mat_mult(A)
print("AB ≠ BA:\nAB:\n", AB, "\nBA:\n", BA, "\n")

AB ≠ BA:
AB:
 12	28
18	42 
BA:
 22	22
32	32 



**Quiz 2**

Write a function make_deck that returns a list of all of the cards in a standard card deck. The return should be a list of tuples of pairs of suit and value. For example the 10 of Clubs would be ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace. Summit your solution with Lab 5

In [47]:
def make_deck():
    suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    values = list(range(2,11)) + ['Ace', 'Jack', 'Queen', 'King'] # range stops at 10

    #create a deck
    deck = [(suit, value) for suit in suits for value in values]
    return deck

In [52]:
deck = make_deck()
print(deck)

[('Hearts', 2), ('Hearts', 3), ('Hearts', 4), ('Hearts', 5), ('Hearts', 6), ('Hearts', 7), ('Hearts', 8), ('Hearts', 9), ('Hearts', 10), ('Hearts', 'Ace'), ('Hearts', 'Jack'), ('Hearts', 'Queen'), ('Hearts', 'King'), ('Diamonds', 2), ('Diamonds', 3), ('Diamonds', 4), ('Diamonds', 5), ('Diamonds', 6), ('Diamonds', 7), ('Diamonds', 8), ('Diamonds', 9), ('Diamonds', 10), ('Diamonds', 'Ace'), ('Diamonds', 'Jack'), ('Diamonds', 'Queen'), ('Diamonds', 'King'), ('Clubs', 2), ('Clubs', 3), ('Clubs', 4), ('Clubs', 5), ('Clubs', 6), ('Clubs', 7), ('Clubs', 8), ('Clubs', 9), ('Clubs', 10), ('Clubs', 'Ace'), ('Clubs', 'Jack'), ('Clubs', 'Queen'), ('Clubs', 'King'), ('Spades', 2), ('Spades', 3), ('Spades', 4), ('Spades', 5), ('Spades', 6), ('Spades', 7), ('Spades', 8), ('Spades', 9), ('Spades', 10), ('Spades', 'Ace'), ('Spades', 'Jack'), ('Spades', 'Queen'), ('Spades', 'King')]
